<a href="https://colab.research.google.com/github/takky0330/Arduino/blob/master/BERT_BunshoAC_IIC_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 乱数シードの固定

import os
import random
import numpy as np
import pandas as pd
import torch
import torchtext  # torchtextを使用

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

In [2]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

In [3]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [0]:
# Livedoorニュースのファイルをダウンロード
# ! wget "https://stream.takky.org/rd/BERT_IIC/2020CS調査_不満_教師.tsv"
# train_data = pd.read_csv("./2020CS調査_不満_教師.csv", encoding='utf-8', sep='\t', lineterminator='\r', index_col=0)

In [5]:
# 教師
train_data = pd.read_csv("https://stream.takky.org/rd/BERT_IIC/2020CS%E8%AA%BF%E6%9F%BB_%E4%B8%8D%E6%BA%80_%E6%95%99%E5%B8%AB.tsv", encoding='utf-8', sep='\t')
train_data

,LABEL,TEXT
0,1,（そこまで大きな不安ではありませんが）初期の対応に少し遅れが出たこと，調査画面の確認が即日で...
1,1,・時々メール返信がない時があり不安になる（大きな不満ではないです）
2,1,お見積り書発行までの時間。（10日程度かかったので、もう少し早いと嬉しいです。）
3,1,サービス全般とよりも，営業担当者の返信が遅すぎた。1週間近く返信が来なかったり，こちらから催...
4,1,メールの反応が著しく遅い時があり、困りました。また以前御社を利用した際にアカデミックで50％...
...,...,...
1496,26,特段なし。
1497,26,特段不満はございません。
1498,26,不満はない。
1499,26,不満点、改善点は今のところございません


In [6]:
# 予測
test_data = pd.read_csv("https://stream.takky.org/rd/BERT_IIC/2020CS%E8%AA%BF%E6%9F%BB_%E4%B8%8D%E6%BA%80_%E4%BA%88%E6%B8%AC.tsv", encoding='utf-8', sep='\t')
test_data

,LABEL,TEXT
0,10,調査パネルと調査会場（新宿）調査画面設計の担当者は対応が早くて満足
1,22,当方より要請した調査票作成対応には満足しております。
2,14,営業担当者の対応が他社に比べれば的確ただ、100％満足かと言われるとそうではない。
3,1,レスポンスや調査スピードが早い。
4,23,スピーディー
...,...,...
1657,22,実査に向けての営業担当の方の対応が丁寧だと思います。
1658,16,わからないことも誠実に丁寧に対応していただけていると感じる。UIも他社と比べるとおそらく使い...
1659,11,回答者のリテラシーが高いこと
1660,25,きちんとデータを出してくれるところ


In [7]:
categories = train_data[train_data.columns.values[0]].unique()
print(categories)
print(len(categories))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26]
26


In [8]:
train_df = pd.DataFrame({'text': train_data[train_data.columns.values[1]].values, 'label': train_data[train_data.columns.values[0]].values})
print(train_df.shape)

(1501, 2)


In [9]:
test_df = pd.DataFrame({'text': test_data[test_data.columns.values[1]].values, 'label': test_data[test_data.columns.values[0]].values})
print(test_df.shape)

(1662, 2)


In [0]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

In [0]:
# DataFrameにカテゴリーindexの列を作成
train_df["label_index"] = train_df["label"].map(dic_cat2id)
test_df["label_index"] = test_df["label"].map(dic_cat2id)

# label列を消去し、text, indexの順番にする
train_df = train_df.loc[:, ["text", "label_index"]]
test_df = test_df.loc[:, ["text", "label_index"]]

In [0]:
# 順番をシャッフルする
train_df = train_df.sample(frac=1, random_state=123).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=123).reset_index(drop=True)

In [13]:
# tsvファイルで保存する

# 訓練&検証データとする
train_df.to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(train_df.shape)

# テストデータとする
test_df.to_csv("./test.tsv", sep='\t', index=False, header=None)
print(test_df.shape)

(1501, 2)
(1662, 2)


In [0]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")

In [15]:
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')

In [0]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定

In [0]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
dataset_train_eval, dataset_test = torchtext.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])

In [18]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

1126
375
1662


In [0]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}

In [20]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
model.eval()
print('ネットワーク設定完了')


ネットワーク設定完了


In [0]:
# BERTでベクトル化する関数を定義


def vectorize_with_bert(net, dataloader):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloader.batch_size

    # データローダーからミニバッチを取り出すループ
    for index, batch in enumerate(dataloader):
        # batchはTextとLableの辞書オブジェクト
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs = batch.Text[0].to(device)  # 文章
        labels = batch.Label.to(device)  # ラベル

        # 順伝搬（forward）計算
        with torch.set_grad_enabled(False):

            # Berに入力
            result = net(inputs)

            # sequence_outputの先頭の単語ベクトルを抜き出す
            vec_0 = result[0]  # 最初の0がsequence_outputを示す
            vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
            vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換

            # ベクトル化したデータをtorchリストにまとめる
            if index == 0:
                list_text = vec_0
                list_label = labels
            else:
                list_text = torch.cat([list_text, vec_0], dim=0)
                list_label = torch.cat([list_label, labels], dim=0)

    return list_text, list_label

In [22]:
# DataLoaderをベクトル化版に変換
# 少し時間がかかります5分弱

list_text_train, list_label_train = vectorize_with_bert(model, dl_train)
list_text_eval, list_label_eval = vectorize_with_bert(model, dl_eval)
list_text_test, list_label_test = vectorize_with_bert(model, dl_test)

使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------
使用デバイス： cuda:0
-----start-------


In [23]:
# torchのリストをDatasetに変換

from torch.utils.data import TensorDataset

dataset_bert_train = TensorDataset(list_label_train.view(-1, 1), list_text_train)
print(len(dataset_bert_train))
dataset_bert_eval = TensorDataset(list_label_eval.view(-1, 1), list_text_eval)
dataset_bert_test = TensorDataset(list_label_test.view(-1, 1), list_text_test)

1126


In [0]:
# Dataloaderにする
from torch.utils.data import DataLoader

# 長すぎるので… batch_size = 1024
batch_size = 64

dl_bert_train = DataLoader(
    dataset_bert_train, batch_size=batch_size, shuffle=True, drop_last=True)
# drop_lastは最後のミニバッチがbatch_sizeに足りない場合は無視する

dl_bert_eval = DataLoader(
    dataset_bert_eval, batch_size=batch_size, shuffle=False)
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=batch_size, shuffle=False)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

OVER_CLUSTRING_RATE = 10


class NetIIC(nn.Module):
    def __init__(self):
        super(NetIIC, self).__init__()

        # multi-headは今回しない
        self.conv1 = nn.Conv1d(1, 400, kernel_size=768, stride=1, padding=0)
        self.bn1 = nn.BatchNorm1d(400)
        self.conv2 = nn.Conv1d(1, 300, kernel_size=400, stride=1, padding=0)
        self.bn2 = nn.BatchNorm1d(300)
        self.conv3 = nn.Conv1d(1, 300, kernel_size=300, stride=1, padding=0)
        self.bn3 = nn.BatchNorm1d(300)

        self.fc1 = nn.Linear(300, 250)
        self.bnfc1 = nn.BatchNorm1d(250)

        # livedoorニュースの9カテゴリに対応するかな？と期待する9分類
        #self.fc2 = nn.Linear(250, 9)
        self.fc2 = nn.Linear(250, len(categories))

        # overclustering
        # 実際の想定よりも多めにクラスタリングさせることで、ネットワークで微細な変化を捉えられるようにする
        #self.fc2_overclustering = nn.Linear(250, 9*OVER_CLUSTRING_RATE)
        self.fc2_overclustering = nn.Linear(250, len(categories)*OVER_CLUSTRING_RATE)

    def forward(self, x):
        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn1(self.conv1(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn2(self.conv2(x)))

        x = x.view(x.size(0), 1, -1)
        x = F.relu(self.bn3(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x_prefinal = F.relu(self.bnfc1(self.fc1(x)))

        # multi-headは使わず
        y = F.softmax(self.fc2(x_prefinal), dim=1)
        y_overclustering = F.softmax(self.fc2_overclustering(
            x_prefinal), dim=1)  # overclustering

        return y, y_overclustering

In [0]:
import torch.nn.init as init


def weight_init(m):
    """重み初期化"""
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        # Xavier
        # init.xavier_normal_(m.weight.data)

        # He
        init.kaiming_normal_(m.weight.data)

        if m.bias is not None:
            init.normal_(m.bias.data)

In [0]:
# IISによる損失関数の定義
# 参考：https://github.com/RuABraun/phone-clustering/blob/master/mnist_basic.py
import sys


def compute_joint(x_out, x_tf_out):
    bn, k = x_out.size()
    assert (x_tf_out.size(0) == bn and x_tf_out.size(1) == k), '{} {} {} {}'.format(
        bn, k, x_tf_out.size(0), x_tf_out.size(1))

    p_i_j = x_out.unsqueeze(2) * x_tf_out.unsqueeze(1)  # bn, k, k
    p_i_j = p_i_j.sum(dim=0)  # k, k
    p_i_j = (p_i_j + p_i_j.t()) / 2.  # symmetrise
    p_i_j = p_i_j / p_i_j.sum()  # normalise
    return p_i_j


def IID_loss(x_out, x_tf_out, EPS=sys.float_info.epsilon):
    # has had softmax applied
    bs, k = x_out.size()
    p_i_j = compute_joint(x_out, x_tf_out)
    assert (p_i_j.size() == (k, k))

    p_i = p_i_j.sum(dim=1).view(k, 1).expand(k, k)
    p_j = p_i_j.sum(dim=0).view(1, k).expand(k, k)

    # avoid NaN losses. Effect will get cancelled out by p_i_j tiny anyway
    # これはPyTorchのバージョン1.3以上だとエラーになる
    # https://discuss.pytorch.org/t/pytorch-1-3-showing-an-error-perhaps-for-loss-computed-from-paired-outputs/68790/3
    #p_i_j[(p_i_j < EPS).data] = EPS
    #p_j[(p_j < EPS).data] = EPS
    #p_i[(p_i < EPS).data] = EPS

    p_i_j = torch.where(p_i_j < EPS, torch.tensor(
        [EPS], device=p_i_j.device), p_i_j)
    p_j = torch.where(p_j < EPS, torch.tensor([EPS], device=p_j.device), p_j)
    p_i = torch.where(p_i < EPS, torch.tensor([EPS], device=p_i.device), p_i)

    # https://qiita.com/Amanokawa/items/0aa24bc396dd88fb7d2a
    # 参考に、重みalphaを追加

    alpha = 2.0
    loss = (- p_i_j * (torch.log(p_i_j) - alpha *
                       torch.log(p_j) - alpha*torch.log(p_i))).sum()

    return loss

In [0]:
# データにノイズを加える関数の定義
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tensor_std = list_text_train.std(dim=0).to(device)


def perturb_data(x):
    y = x.clone()
    noise = torch.randn(len(tensor_std)).to(device)*tensor_std*2.0
    noise = noise.expand(x.shape[0], -1)
    y += noise

    return y

In [0]:
# 学習関数の定義

def train(total_epoch, model, train_loader, optimizer, device):

    # ネットワークを訓練モードに
    model.train()

    # 学習率のスケジューラーCosAnnealing
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=2, T_mult=2, eta_min=0)

    for epoch in range(total_epoch):
        for batch_idx, (target, data) in enumerate(train_loader):

            # 学習率変化
            scheduler.step()
            
            data_perturb = perturb_data(data)  # ノイズを与え、変換したデータを作る

            # GPUに送れる場合は送る
            data = data.to(device)
            data_perturb = data_perturb.to(device)

            # 最適化関数の初期化
            optimizer.zero_grad()

            # ニューラルネットワークへ入れる
            output, output_overclustering = model(data)
            output_perturb, output_perturb_overclustering = model(data_perturb)

            # 損失の計算
            loss1 = IID_loss(output, output_perturb)
            loss2 = IID_loss(output_overclustering, output_perturb_overclustering)
            loss = loss1 + loss2

            # 損失を減らすように更新
            loss.backward()
            optimizer.step()

        # ログ出力
        if epoch % 50 == 0:
            print('Train Epoch {} \tLoss1: {:.6f} \tLoss2: {:.6f} \tLoss_total: {:.6f}'.format(
                epoch, loss1.item(), loss2.item(), loss1.item()+loss2.item()))

    return model, optimizer

In [30]:
# 学習の実施(5分弱)

# モデルの用意
net = NetIIC()
net.apply(weight_init)
net.to(device)

# 最適化関数
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4) 

total_epoch = 3000

model_trained, optimizer = train(
    total_epoch, net, dl_bert_train, optimizer, device)

Train Epoch 0 	Loss1: -5.882688 	Loss2: -10.548670 	Loss_total: -16.431357
Train Epoch 50 	Loss1: -8.889074 	Loss2: -13.265369 	Loss_total: -22.154444
Train Epoch 100 	Loss1: -9.040634 	Loss2: -13.458563 	Loss_total: -22.499197
Train Epoch 150 	Loss1: -9.094157 	Loss2: -13.618498 	Loss_total: -22.712655
Train Epoch 200 	Loss1: -8.692152 	Loss2: -13.246690 	Loss_total: -21.938842
Train Epoch 250 	Loss1: -8.880647 	Loss2: -13.552317 	Loss_total: -22.432963
Train Epoch 300 	Loss1: -9.087015 	Loss2: -13.699569 	Loss_total: -22.786584
Train Epoch 350 	Loss1: -9.228639 	Loss2: -13.845385 	Loss_total: -23.074023
Train Epoch 400 	Loss1: -9.045626 	Loss2: -13.668970 	Loss_total: -22.714596
Train Epoch 450 	Loss1: -8.939167 	Loss2: -13.484238 	Loss_total: -22.423405
Train Epoch 500 	Loss1: -9.021594 	Loss2: -13.449301 	Loss_total: -22.470895
Train Epoch 550 	Loss1: -9.381926 	Loss2: -13.983746 	Loss_total: -23.365671
Train Epoch 600 	Loss1: -9.266672 	Loss2: -13.875395 	Loss_total: -23.142067
Tr

In [0]:
# モデル分類のクラスターの結果を確認する
import numpy as np

# ミニバッチサイズ1のテスト用のDataLoaderを用意
dl_bert_test = DataLoader(
    dataset_bert_test, batch_size=1, shuffle=False)


def test(model, device, test_loader):
    model.eval()

    out_targs = []
    ref_targs = []

    # 出力用のリストを用意
    total_num = len(test_loader)
    # index, (target_label, inferenced_label)
    output_list = np.zeros((total_num, 2))

    with torch.no_grad():
        for batch_idx, (target, data) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            outputs, outputs_overclustering = model(data)

            # 分類結果をリストに追加
            out_targs.append(outputs.argmax(dim=1).cpu())
            ref_targs.append(target[0].cpu())

            # 結果をリストにまとめる
            output_list[batch_idx, 0] = target[0][0].cpu()  # 正解ラベル
            output_list[batch_idx, 1] = outputs.argmax(dim=1).cpu()  # 予測ラベル

    out_targs = torch.cat(out_targs)
    ref_targs = torch.cat(ref_targs)

    return out_targs.view(-1, 1).numpy(), ref_targs.numpy(), output_list

In [32]:
# テストデータで推論を実施
out_targs, ref_targs, output_list = test(model_trained, device, dl_bert_test)
print(len(dl_bert_test))

1662


In [33]:
# 混同行列（的な）を作る
#matrix = np.zeros((9, 9))
matrix = np.zeros((len(categories), len(categories)))

# 縦にlivedoorニュースの正解クラスを、横に判定されたクラスの頻度表を作成
for i in range(len(out_targs)):
    row = ref_targs[i]
    col = out_targs[i]
    matrix[row][col] += 1

np.set_printoptions(suppress=True)
print(matrix)

[[ 0.  6.  1.  0.  5.  9.  5.  2.  7.  0.  5.  8. 17.  6.  2.  0.  5. 16.
   0.  1.  7.  2.  9.  0.  1.  0.]
 [ 5. 13.  2.  0.  6.  6. 12.  0.  8.  6.  9.  7. 19. 13. 12.  0. 10.  7.
   3.  2.  3.  5.  8.  0.  9.  3.]
 [ 0.  3.  3.  0.  0.  2.  4.  0.  4.  5.  0.  1.  2.  1.  7.  0.  3.  3.
   2.  3.  0.  4.  0.  0.  3.  1.]
 [ 0.  0.  1.  0.  1.  0.  0.  0.  0.  2.  0.  0.  0.  0.  1.  0.  0.  0.
   0.  1.  0.  0.  0.  0.  0.  0.]
 [ 2. 12. 10.  0. 16.  1. 10.  0. 10.  7.  2.  8. 10.  8. 12.  0.  8.  1.
   2.  4.  1.  7.  6.  0.  9.  2.]
 [ 1.  0.  0.  0.  2.  0.  2.  0.  6.  0.  0.  0.  2.  0.  1.  0.  2.  2.
   0.  1.  0.  1.  3.  0.  2.  0.]
 [ 0.  2.  0.  0.  1.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  4.  2.  0.  4.  1.  2.  1.  3.  1.  1.  2.  8.  0.  3.  0.  1.  1.
   0.  0.  0.  1.  1.  0.  1.  1.]
 [ 2.  1.  0.  0.  2.  0.  3.  0.  1.  1.  0.  0.  3.  2.  1.  0.  1.  0.
   1.  0.  0.  1.  0.  0.  3.  0.]
 [ 0.  8.  4.  0. 1

In [0]:
# クラスタの結果を確認
#「sports-watch」の5番目のクラスタの文章、7番目のクラスタの文章
#「topic-news」の5番目のクラスタの文章、7番目のクラスタの文章
# を確認して、クラスタ5とクラスタ7の特徴を見てみます。

import pandas as pd

df2 = pd.DataFrame(output_list)
df2.columns=["正解クラス", "推定クラスタ"]

In [0]:
df_test_txt = pd.read_csv('test.tsv', encoding='utf-8', sep='\t', header=None)
df_test_txt.columns=["TEXT", "CLASS"]

In [0]:
result_df = pd.concat([df2, df_test_txt["TEXT"]], axis=1)
result_df.to_csv("./BERT_BunshoAC_IIC_2.tsv", encoding='utf-8', sep='\t')

In [0]:
from google.colab import files
files.download("./BERT_BunshoAC_IIC_2.tsv")

In [53]:
result_df[result_df["推定クラスタ"] == 1.]

,正解クラス,推定クラスタ,TEXT
8,2.0,1.0,当方が実施したい調査目的などを理解していただいた上で、専門的な助言をいただける事
14,21.0,1.0,・CLTの実査の質は高く、正確かつ慎重に実施いただいていると思います。
17,10.0,1.0,モニターの回収がしっかりできること、また簡易的な調査向けに様々なサービスを設けてくださってい...
33,21.0,1.0,こちらのリサーチ経験や知見がない中で、快く相談に乗ってくださりとても心強かったです。
39,4.0,1.0,質問内容・方法の設定含む調査票作成・集計に就いて素晴らしいと感じました。
...,...,...,...
1587,11.0,1.0,発注～納品までのスピード、モニタ数の多さに特に満足しています。
1589,13.0,1.0,・営業担当の方の対応が早くて助かっています。
1621,1.0,1.0,調査案件内容に合わせて、打ち合わせをして構築できるので安心します。
1630,1.0,1.0,調査票作成について、柔軟かつ迅速に対応していただいた。


In [47]:
result_df[result_df["推定クラスタ"] == 2.]

,正解クラス,推定クラスタ,TEXT
42,13.0,2.0,お見積りや設計等、何度も変更することがあるのですが、営業の方は毎回迅速に対応してくださる点。...
64,4.0,2.0,納品までの早さ。また、回収段階の時点で速報数値が確認できる点については作業がしやすく満足して...
101,9.0,2.0,いくつかのリサーチ会社様と業務を行って参りましたが、モニタ規模とスピード感は断トツだと感じて...
112,12.0,2.0,集計ソフトが使いにくい。そもそもパソコンによっては動かなかったり、問合せしても返事がなかった...
137,13.0,2.0,他のネット系リサーチ会社に対して、総合的に安心してお任せできる。調査画面の確認など、精緻にチ...
221,12.0,2.0,クイッククロスで簡単にクロス集計ができるので助かります。複雑なクロスの場合には、担当者の人が...
269,15.0,2.0,パネルの分厚さには非常に満足しています。我々のユーザー（登録者、ファン）ではカバーできない一...
284,7.0,2.0,与件内容をお伝えした際に、内容を汲み取り、調査票設計時にご意見をいただけたり、レポーティング...
311,9.0,2.0,いつもふわっとした状態での依頼なうえ、ナルハヤでの納品を希望することが多くお手数をおかけして...
401,4.0,2.0,何と言ってもアナリストのアドバイスの的確さ、提案の精度の高さです。別の調査会社とも取引した経...
